# Big Data for Engineers – Exercises 

# Spring 2023 -  Week 11 - Rumble



# 0. RumbleSandbox

To play with simple JSONiq queries without installing RumbleDB, you can use the [RumbleDB sandbox](https://colab.research.google.com/github/RumbleDB/rumble/blob/master/RumbleSandbox.ipynb#scrollTo=u38jSNpB--QW).\
The sandbox uses a small public backend, so please install Rumble on your local machine for the rest of the exercise.

# 1. Install Java and Download Rumble
You might have noticed that the process is the same as in section 5 of Exercise06. BUT, a newer version of RumbleDB has been released since then :) Please download the latest RumbleDB 1.21.0 for this exercise.

## Option 1: Locally

To install the latest version of RumbleDB locally, you can follow the steps given on the official rumbledb website: 
https://rumble.readthedocs.io/en/latest/Getting%20started/

Please follow very carefully the instructions of "Method 1: with the large, standalone, RumbleDB jar (experimental)". In particular,
1. Download and install Java 8 or 11. Make sure that the Java version is correct by running `java -version` in the terminal.
2. Download the rumbledb-1.21.0-standalone.jar. You can put the jar file in the working directory of this exercise.

You also need to correctly set the `JAVA_HOME` environment variable. Please do this everytime you open a new terminal if you don't change the Bash initialization file. You can verify by running `echo $JAVA_HOME`. 

For example, on MacOS, for Java 11, (1) you can `export JAVA_HOME=$(/usr/libexec/java_home -v 11)`; or (2) `export JAVA_HOME=/Library/Java/JavaVirtualMachines/jdk-11.0.1.jdk/Contents/Home/` or a similar path that exists on your machine. \
On Linux/Windows OS there are similar methods to set `JAVA_HOME`.\


## Option 2: Github codespaces
If the local installation does not work for you, please use this option. See this [tutorial](https://cloud.inf.ethz.ch/s/KHLnq9HsQHfLHHQ/download) for running RumbleDB on Github codespaces.

## JSONiq shell
To launch a JSONiq shell, run:

`java -jar rumbledb-1.21.0-standalone.jar repl`

This gives access to the RumbleDB shell where you can try commands. Note that to run each command, you should press 'Enter' twice, and you can quit with 'exit'.

Feel free to try the many functions available, here is a link with the exhaustive list: https://rumble.readthedocs.io/en/latest/Function%20library/.

# 2. Setup Rumble in Jupyter Notebook



### Install Jupyter Notebook



In order to execute the queries in this notebook, you need to [install](https://jupyter.org/install) jupyter notebook on your **own machine**, and then download this notebook and [run](https://jupyter.readthedocs.io/en/latest/running.html#running) it locally rather than rely on the colab.

From now on, we want to run the code on this notebook instead of the shell, hence you can close the shell (Ctrl+C or type 'exit') and execute your own Rumble server by running:

`java -jar rumbledb-1.21.0-standalone.jar --server yes --port 9090`

The mount allows RumbleDB to directly read the files from your local disk, in this case your exercise folder of this week.

It is recommended to use the port 9090 which normally does not get occupied by other applications.

To verify that the RumbleDB is up and running, you can go to http://localhost:9090/public.html in a web browser and you can type queries there.

Then, you need to execute the cell below to activate the jsoniq magic (you do not need to understand what it does, this is just initialization Python code).

In [ ]:
!pip install rumbledb
%load_ext rumbledb
%env RUMBLEDB_SERVER=http://localhost:9090/jsoniq

Now we are all set! You can now start reading and executing the JSONiq queries in this notebook as you go, and you can even edit them!

# 3. JSONiq

## JSON

As explained on the [official JSON Web site](http://www.json.org/), JSON is a lightweight data-interchange format designed for humans as well as for computers. It supports as values:
- objects (string-to-value maps)
- arrays (ordered sequences of values)
- strings
- numbers
- booleans (true, false)
- null

JSONiq provides declarative querying and updating capabilities on JSON data.

## Elevator Pitch

JSONiq is based on XQuery, which is a W3C standard (like XML and HTML). XQuery is a very powerful declarative language that originally manipulates XML data, but it turns out that it is also a very good fit for manipulating JSON natively.
JSONiq, since it extends XQuery, is a very powerful general-purpose declarative programming language. Our experience is that, for the same task, you will probably write about 80% less code compared to imperative languages like JavaScript, Python or Ruby. Additionally, you get the benefits of strong type checking without actually having to write type declarations.
Here is an appetizer before we start the tutorial from scratch.


In [ ]:
%%jsoniq

let $stores :=
[
  { "store number" : 1, "state" : "MA" },
  { "store number" : 2, "state" : "MA" },
  { "store number" : 3, "state" : "CA" },
  { "store number" : 4, "state" : "CA" }
]
let $sales := [
   { "product" : "broiler", "store number" : 1, "quantity" : 20  },
   { "product" : "toaster", "store number" : 2, "quantity" : 100 },
   { "product" : "toaster", "store number" : 2, "quantity" : 50 },
   { "product" : "toaster", "store number" : 3, "quantity" : 50 },
   { "product" : "blender", "store number" : 3, "quantity" : 100 },
   { "product" : "blender", "store number" : 3, "quantity" : 150 },
   { "product" : "socks", "store number" : 1, "quantity" : 500 },
   { "product" : "socks", "store number" : 2, "quantity" : 10 },
   { "product" : "shirt", "store number" : 3, "quantity" : 10 }
]
let $join :=
  for $store in $stores[], $sale in $sales[]
  where $store."store number" = $sale."store number"
  return {
    "nb" : $store."store number",
    "state" : $store.state,
    "sold" : $sale.product
  }
return [$join]

## And here you go

### Actually, you already knew some JSONiq

The first thing you need to know is that a well-formed JSON document is a JSONiq expression as well.
This means that you can copy-and-paste any JSON document into a query. The following are JSONiq queries that are "idempotent" (they just output themselves):

In [ ]:
%%jsoniq
{ "pi" : 3.14, "sq2" : 1.4 }

In [ ]:
%%jsoniq
[ 2, 3, 5, 7, 11, 13 ]

In [ ]:
%%jsoniq
{
      "operations" : [
        { "binary" : [ "and", "or"] },
        { "unary" : ["not"] }
      ],
      "bits" : [
        0, 1
      ]
    }

In [ ]:
%%jsoniq
[ { "Question" : "Ultimate" }, ["Life", "the universe", "and everything"] ]


This works with objects, arrays (even nested), strings, numbers, booleans, null.

It also works the other way round: if your query outputs an object or an array, you can use it as a JSON document. JSONiq is a declarative language. This means that you only need to say what you want - the compiler will take care of the how.

In the above queries, you are basically saying: I want to output this JSON content, and here it is.

## JSONiq basics

### The real JSONiq Hello, World!

Wondering what a hello world program looks like in JSONiq? Here it is:

In [ ]:
%%jsoniq
"Hello, World!"

Not surprisingly, it outputs the string "Hello, World!".

### Numbers and arithmetic operations

Okay, so, now, you might be thinking: "What is the use of this language if it just outputs what I put in?" Of course, JSONiq can more than that. And still in a declarative way. Here is how it works with numbers:

In [ ]:
%%jsoniq
2 + 2

In [ ]:
%%jsoniq
 (38 + 2) div 2 + 11 * 2


(mind the division operator which is the "div" keyword. The slash operator has different semantics).

Like JSON, JSONiq works with decimals and doubles:

In [ ]:
%%jsoniq
 6.022e23 * 42

### Logical operations

JSONiq supports boolean operations.

In [ ]:
%%jsoniq
true and false

In [ ]:
%%jsoniq
(true or false) and (false or true)

The unary not is also available:

In [ ]:
%%jsoniq
not true

### Strings

JSONiq is capable of manipulating strings as well, using functions:


In [ ]:
%%jsoniq
concat("Hello ", "Captain ", "Kirk")

In [ ]:
%%jsoniq
substring("Mister Spock", 8, 5)

JSONiq comes up with a rich string function library out of the box, inherited from its base language. These functions are listed [here](https://www.w3.org/TR/xpath-functions-30/) (actually, you will find many more for numbers, dates, etc).



### Sequences

Until now, we have only been working with single values (an object, an array, a number, a string, a boolean). JSONiq supports sequences of values. You can build a sequence using commas:


In [ ]:
%%jsoniq
 (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [ ]:
%%jsoniq
1, true, 4.2e1, "Life"

The "to" operator is very convenient, too:

In [ ]:
%%jsoniq
 (1 to 100)

Some functions even work on sequences:

In [ ]:
%%jsoniq
sum(1 to 100)

In [ ]:
%%jsoniq
string-join(("These", "are", "some", "words"), "-")

In [ ]:
%%jsoniq
count(10 to 20)

In [ ]:
%%jsoniq
avg(1 to 100)

Unlike arrays, sequences are flat. The sequence (3) is identical to the integer 3, and (1, (2, 3)) is identical to (1, 2, 3).

## A bit more in depth

### Variables

You can bind a sequence of values to a (dollar-prefixed) variable, like so:

In [ ]:
%%jsoniq
let $x := "Bearing 3 1 4 Mark 5. "
return concat($x, "Engage!")

In [ ]:
%%jsoniq
let $x := ("Kirk", "Picard", "Sisko")
return string-join($x, " and ")

You can bind as many variables as you want:

In [ ]:
%%jsoniq
let $x := 1
let $y := $x * 2
let $z := $y + $x
return ($x, $y, $z)

and even reuse the same name to hide formerly declared variables:

In [ ]:
%%jsoniq
let $x := 1
let $x := $x + 2
let $x := $x + 3
return $x

### Iteration

In a way very similar to let, you can iterate over a sequence of values with the "for" keyword. Instead of binding the entire sequence of the variable, it will bind each value of the sequence in turn to this variable.

In [ ]:
%%jsoniq
for $i in 1 to 10
return $i * 2

More interestingly, you can combine fors and lets like so:

In [ ]:
%%jsoniq
let $sequence := 1 to 10
for $value in $sequence
let $square := $value * 2
return $square

and even filter out some values:

In [ ]:
%%jsoniq
let $sequence := 1 to 10
for $value in $sequence
let $square := $value * 2
where $square < 10
return $square

Note that you can only iterate over sequences, not arrays. To iterate over an array, you can obtain the sequence of its values with the [] operator, like so:


In [ ]:
%%jsoniq
[1, 2, 3][]

### Conditions

You can make the output depend on a condition with an if-then-else construct:

In [ ]:
%%jsoniq
for $x in 1 to 10
return if ($x < 5) then $x
                   else -$x

Note that the else clause is required - however, it can be the empty sequence () which is often when you need if only the then clause is relevant to you.

### Composability of Expressions

Now that you know of a couple of elementary JSONiq expressions, you can combine them in more elaborate expressions. For example, you can put any sequence of values in an array:

In [ ]:
%%jsoniq
[ 1 to 10 ]

Or you can dynamically compute the value of object pairs (or their key):

In [ ]:
%%jsoniq
{
      "Greeting" : (let $d := "Mister Spock"
                    return concat("Hello, ", $d)),
      "Farewell" : string-join(("Live", "long", "and", "prosper"),
                               " ")
}

You can dynamically generate object singletons (with a single pair):


In [ ]:
%%jsoniq
{ concat("Integer ", 2) : 2 * 2 }

and then merge lots of them into a new object with the {| |} notation:

In [ ]:
%%jsoniq
{|
    for $i in 1 to 10
    return { concat("Square of ", $i) : $i * $i }
|}

## JSON Navigation

Up to now, you have learnt how to compose expressions so as to do some computations and to build objects and arrays. It also works the other way round: if you have some JSON data, you can access it and navigate.
All you need to know is: JSONiq views
an array as an ordered list of values,
an object as a set of name/value pairs


### Objects

You can use the dot operator to retrieve the value associated with a key. Quotes are optional, except if the key has special characters such as spaces. It will return the value associated thereto:

In [ ]:
%%jsoniq
let $person := {
    "first name" : "Sarah",
    "age" : 13,
    "gender" : "female",
    "friends" : [ "Jim", "Mary", "Jennifer"]
}
return $person."first name"

You can also ask for all keys in an object:

In [ ]:
%%jsoniq
let $person := {
    "name" : "Sarah",
    "age" : 13,
    "gender" : "female",
    "friends" : [ "Jim", "Mary", "Jennifer"]
}
return { "keys" : [ keys($person)] }

### Arrays

The [[]] operator retrieves the entry at the given position:

In [ ]:
%%jsoniq
let $friends := [ "Jim", "Mary", "Jennifer"]
return $friends[[1+1]]

It is also possible to get the size of an array:

In [ ]:
%%jsoniq
let $person := {
    "name" : "Sarah",
    "age" : 13,
    "gender" : "female",
    "friends" : [ "Jim", "Mary", "Jennifer"]
}
return { "how many friends" : size($person.friends) }

Finally, the [] operator returns all elements in an array, as a sequence:

In [ ]:
%%jsoniq
let $person := {
    "name" : "Sarah",
    "age" : 13,
    "gender" : "female",
    "friends" : [ "Jim", "Mary", "Jennifer"]
}
return $person.friends[]

### Relational Algebra

Do you remember SQL's SELECT FROM WHERE statements? JSONiq inherits selection, projection and join capability from XQuery, too.

In [ ]:
%%jsoniq
let $stores :=
[
    { "store number" : 1, "state" : "MA" },
    { "store number" : 2, "state" : "MA" },
    { "store number" : 3, "state" : "CA" },
    { "store number" : 4, "state" : "CA" }
]
let $sales := [
    { "product" : "broiler", "store number" : 1, "quantity" : 20  },
    { "product" : "toaster", "store number" : 2, "quantity" : 100 },
    { "product" : "toaster", "store number" : 2, "quantity" : 50 },
    { "product" : "toaster", "store number" : 3, "quantity" : 50 },
    { "product" : "blender", "store number" : 3, "quantity" : 100 },
    { "product" : "blender", "store number" : 3, "quantity" : 150 },
    { "product" : "socks", "store number" : 1, "quantity" : 500 },
    { "product" : "socks", "store number" : 2, "quantity" : 10 },
    { "product" : "shirt", "store number" : 3, "quantity" : 10 }
]
let $join :=
    for $store in $stores[], $sale in $sales[]
    where $store."store number" = $sale."store number"
    return {
        "nb" : $store."store number",
        "state" : $store.state,
        "sold" : $sale.product
    }
return [$join]

### Access datasets

Rumble can read input from many file systems and many file formats. If you are using our backend, you can only use json-doc() with any URI pointing to a JSON file and navigate it as you see fit. 

In [ ]:
%%jsoniq
json-doc("Put any HTTP URL pointing to a JSON document here!").foo[[1]].bar.foobar[]

If you are using your own Rumble server on your cluster, you can also use any other function and scheme.

In [ ]:
%%jsoniq
json-file("put the path to a JSON lines file here. This will only work against your own Rumble backend and Spark cluster, though.")

# 4. The Great Language Game

This week you will be using again the [language confusion dataset](https://quietlyamused.org/blog/2014/03/12/language-confusion/). You will write queries with Rumble.

Download and decompress the dataset in the same folder as `rumbledb-1.21.0.jar` with the following:
```
wget https://cloud.inf.ethz.ch/s/a8FoHew6dHKGYKK/download/confusion20140302.tbz2
tar -jxvf confusion20140302.tbz2
```
or, if `wget` is not found,
```
curl -O https://cloud.inf.ethz.ch/s/a8FoHew6dHKGYKK/download/confusion20140302.tbz2
tar -jxvf confusion20140302.tbz2
```

Now, you should have a folder with a (large) json file inside.

## 4.0 Check the data

We provide you with two datasets, you can already check that you get the correct number of records. The full dataset should contain 16511224 records. We also provided you with a second small dataset to test your queries faster, it contains 100000 records.

In [ ]:
%%jsoniq
count(for $i in json-file("confusion-2014-03-02/confusion-2014-03-02.json")
return $i)

In [ ]:
%%jsoniq
count(for $i in json-file("confusion-100000.json")
return $i)

Print one record

In [ ]:
%%jsoniq
json-file("confusion-100000.json")[1]

List the keys

In [ ]:
%%jsoniq
keys(json-file("confusion-100000.json"))

## 4.1 Query the data

You can read data from a json file with `json-file`. For example, the following query will read and print the entries in the confusion dataset:




In [ ]:
%%jsoniq
for $i in json-file("confusion-100000.json")
return $i

After the results of your query are printed, Rumble will report the execution runtime in milliseconds:
```
Took: 62.02618598937988 ms
```

In the `json-file` method you can optionally specify the number of partitions, which may allow your query to be parallelized and executed faster. This offers great advantage on cluster, but is not much faster locally. For example:


In [ ]:
%%jsoniq
for $i in json-file("confusion-100000.json", 10)
return $i

## 4.2 SQL to Rumble

The following examples, show how SQL queries can be converted to JSONiq queries. Assume that the dataset is accessible with SQL through the table "entries".





### 4.2.1 Get all games played from Switzerland


```sql
SELECT *
FROM entries
WHERE country == "CH"
```


In [ ]:
%%jsoniq
for $i in json-file("confusion-100000.json", 10)
where $i.country eq "CH"
return $i

### 4.2.2 Get all games played from Switzerland, where the correct answer (target) was "German"
```sql
SELECT *
FROM entries
WHERE country == "CH" AND target == "German"
```




In [ ]:
%%jsoniq
for $i in json-file("confusion-100000.json", 10)
where $i.country eq "CH" and $i.target eq "German"
return $i

### 4.2.3 Get the top 5 games played from Switzerland, where the correct answer (target) was "German"
```sql
SELECT *
FROM entries
WHERE country == "CH" AND target == "German"
LIMIT 5
```




In [ ]:
%%jsoniq
let $ch-de_data := for $i in json-file("confusion-100000.json", 10)
    where $i.country eq "CH" and $i.target eq "German"
    return $i
for $j in $ch-de_data
count $c
where $c <= 5
return $j

### 4.2.4 Get all games played from Switzerland, where the correct answer (target) was "German", order them by date (ascending), and return the top 5 rows.
```sql
SELECT *
FROM entries
WHERE country == "CH" AND target == "German"
ORDER BY date ASC
LIMIT 5
```




In [ ]:
%%jsoniq
let $ch-de_data := for $i in json-file("confusion-100000.json", 10)
    order by $i.date ascending
    return $i
for $j in $ch-de_data
count $c
where $c <= 5
return $j

### 4.2.5 Get all games played from Switzerland, where the correct answer (target) was "German", group them by date, and return for each different date the number of games played.

```sql
SELECT date, COUNT(*) AS num_games
FROM entries
WHERE country == "CH" AND target == "German"
GROUP BY date
```


In [ ]:
%%jsoniq
for $i in json-file("confusion-100000.json", 10)
where $i.country eq "CH" and $i.target eq "German"
group by $d := $i.date
return {"date": $d, "num_games": count($i)}

### 4.2.6 Get all games played from Switzerland, group them by date and target, and return for each different date and target the number of games played.


NOTE: Rumble has some reserved keywords, for example `date`. If you try to create a variable `$date`, you may get an error, such as `no viable alternative at input 'date'`.

```sql
SELECT date, target, COUNT(*) AS num_games
FROM entries
WHERE country == "CH"
GROUP BY date, target
```




In [ ]:
%%jsoniq
for $i in json-file("confusion-100000.json", 10)
where $i.country eq "CH"
group by $d := $i.date, $target := $i.target
return {"date": $d, "target": $target, "num_games": count($i)}


### 4.2.7 For all games played from Switzerland, return the distinct targets of those games.

```sql
SELECT DISTINCT(target)
FROM entries
WHERE country == "CH"
```




In [ ]:
%%jsoniq
let $targets := for $i in json-file("confusion-100000.json", 10)
  where $i.country eq "CH"
  return $i.target
return distinct-values($targets)

### 4.2.8 For all games played from Switzerland, get the distinct targets of those games, and return the index of "German" in the list of distinct targets.




In [ ]:
%%jsoniq
let $targets := for $i in json-file("confusion-100000.json", 10)
    where $i.country eq "CH"
    return $i.target
return index-of(distinct-values($targets), "German")


### 4.2.9 Count the number of games played from Switzerland (without any grouping).


NOTE: `distinct-values` and `index-of` work on "sequences". The method `json-file` returns a sequence. If you have an array on which you want to apply `distinct-values` and `index-of`, you must first convert it to a sequence. This can be done with `[]`. For example, if you have an array called `arr`, you can find its distinct values with `distinct-values(arr[])`

```sql
SELECT COUNT(*) AS count
FROM entries
WHERE country == "CH"
```


In [ ]:
%%jsoniq
count(for $i in json-file("confusion-100000.json", 10)
    where $i.country eq "CH"
    return $i
)

If in your query you want to join 2 (or more) sequences (results of `json-file` or subqueries), you can do it in the following way:
```
let $seq1 := ...
let $seq2 := ...
for $i in $seq1, $j in $seq2
where $i.attr1 eq $j.attr2
...
```

## 4.3 More queries

Try writing a few more queries:
- List all chosen answers to games where the guessed language is correct (=target).

In [ ]:
%%jsoniq

- Count the games where the index of the correct answer in the choices array is 2 (as returned by the index-of method).

In [ ]:
%%jsoniq

- Return all games played on September 1st 2013.

In [ ]:
%%jsoniq

# 5. More nestedness
## 5.1 Create Nestedness
You may remember in the exercise of Spark Dataframes & Spark SQL, we mentioned two methods <font face="courier">collect_set/collect_list</font> for creating arrays. In JSONiq, this kind of things become even simpler because JSONiq natively supports JSON, so we can directly create arrays via adding square brackets (<font face="courier">[]</font>) and even without any group by operations. 

For example, if we want to know the list of date at which "Fijian" was used as the target, we can write a simple JSONiq query:

In [ ]:
%%jsoniq
let $dateSeq := for $i in json-file("confusion-100000.json", 10)
    where $i.target eq "Fijian"
    return $i.date
return [subsequence($dateSeq, 0, 10)]

The above query is basically a counterpart of a Spark Dataframe query with <font face="courier">collect_list</font>. If we want to imitate the behavior of <font face="courier">collect_set</font>, which means we want the result array to be de-duplicated, we can just resort to <font face="courier">distinct-values</font>:

In [ ]:
%%jsoniq
let $dateSeq := for $i in json-file("confusion-100000.json", 10)
where $i.target eq "Fijian"
return $i.date
return [distinct-values($dateSeq)]

Now what if we want to know for each different language, the de-duplicated list of date at which it was used as a target? We may need group by again. Try to come up with the query on your own. What might be the difference in the query with and without group by?

**Note:** Do not hesitate to only use the small provided dataset because the query on the original dataset might take very long and consume a humongous amount of memory. The most important thing is not about the answer, but to come up with proper queries.

In [ ]:
%%jsoniq
for $i in json-file("confusion-100000.json", 10) 
group by $j := $i.target
return {"language":$j, "dates":[distinct-values($i.date)]}

The difference is, if there's group by in a query, after group by, the loop variable (\$i in the above query) will no longer be a single entry, but instead bound to the **sequence** corresponding to each group, which means we can directly apply functions like distinct-values to \$i.
In the query above without group by, \$i always corresponds to one entry, so there's no point to do things like distinct-values(\$i.date). This is why we need to get the intermediate result $dataSeq, and then create arrays and apply distinct-values on this whole sequence.

Obviously, unlike <font face="courier">collect_set/collect_list</font> which only accept one column and create arrays on that very column, JSONiq can create arrays on arbitrary things. For example, if we want to highlight the information of date, we can create a new dataset that shows the game info for each date:

In [ ]:
%%jsoniq
let $newDataset := for $i in json-file("confusion-100000.json", 10) 
  group by $d := $i.date
  return {"date": $d, "info": [$i]}
return $newDataset

Now we have a more nested dataset! We can try to redo some of the exercises above with this new dataset. For example, get all games played in Switzerland:

In [ ]:
%%jsoniq


Try another one: get the count of games played in Switzerland:

In [ ]:
%%jsoniq


Feel free to try out more questions on your own!

## 5.2 Git-archive dataset
Now let's get into the mess of the real world. We are going to explore the git-archive dataset to handle some properly messy data, which is very challenging if you use Spark. 

For getting the dataset, just run:
```
wget https://polybox.ethz.ch/index.php/s/HVWlvJAXVkQ05cw/download -O git-archive.json
```
or use curl as done for the great game language dataset.

Have a look at what the dataset looks like:

In [ ]:
%%jsoniq
for $i in json-file("git-archive.json", 10)
count $c
where $c <= 1
return $i

What a mess, isn't it? Anyway, let's try to write some challenging queries, since you've already mastered those easy ones with the language game dataset...

1. What is the number of distinct author names that are part of a push event (i.e., an event with the type PushEvent)?

In [ ]:
%%jsoniq


2. What is the name of the repository with the highest number of push events (i.e., events with the type PushEvent), and how many of these push events occurred in this repository?

In [ ]:
%%jsoniq


## BONUS: Machine Learning in Rumble

Not only RumbleDB allows to build the entire data pipeline by querying large messy datasets and clean, structure, normoalize and validate them but now also supports feeding the data into machine learning models to build an all-in-one solution. RumbleDB uses Spark to distribute computations accross a cluster, and for ML it is also based on the SparkML library (https://spark.apache.org/docs/latest/ml-guide.html). Here is a simple example to clusterize data points based on a KMeans model.

In the following example we declare a user defined type col-1-2 with an id and two decimal columns. We then write our data in a variable local-data, which we validate against our defined type and then forward the data to a transformer. Finally, we instantiate a K-means estimator to train our data on it.

In [ ]:
%%jsoniq
declare type local:col-1-2 as {
  "id": "integer",
  "col1": "decimal",
  "col2": "decimal"
};

let $vector-assembler := get-transformer("VectorAssembler")(
  ?,
  { "inputCols" : [ "col1", "col2" ], "outputCol" : "features" }
)

let $local-data := (
    {"id": 0, "col1": 0.0, "col2": 3.0},
    {"id": 1, "col1": 2.1, "col2": 0.1},
    {"id": 2, "col1": 0.2, "col2": 0.2},
    {"id": 3, "col1": 6.0, "col2": 9.0},
    {"id": 4, "col1": 5.1, "col2": 9.1},
    {"id": 5, "col1": 9.2, "col2": 9.2}
)
let $df-data := validate type local:col-1-2* {$local-data }
let $df-data := $vector-assembler($df-data)

let $est := get-estimator("KMeans")
let $tra := $est(
    $df-data,
    {"featuresCol": "features"}
)

for $i in $tra(
    $df-data,
    {"featuresCol": "features"}
)
return $i

### Visualize the output

Install `pandas` and `matplotlib` if you haven't:
`pip install pandas matplotlib`

In [ ]:
import pandas as pd

d = [{"id": 0, "col1": 0.0, "col2": 3.0, "prediction": 0},
    {"id": 1, "col1": 2.1, "col2": 0.1, "prediction": 0},
    {"id": 2, "col1": 0.2, "col2": 0.2, "prediction": 0},
    {"id": 3, "col1": 6.0, "col2": 9.0, "prediction": 1},
    {"id": 4, "col1": 5.1, "col2": 9.1, "prediction": 1},
    {"id": 5, "col1": 9.2, "col2": 9.2, "prediction": 1}]

df = pd.DataFrame(d)
print (df)

ax1 = df.plot.scatter(x='col1',
                      y='col2',
                      c='prediction', cmap='winter')